In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# /kaggle/input/car-crash-dataset-ccd/CrashBest
# /kaggle/input/car-crash-dataset-ccd/CrashBest/C_000001_01.jpg
# /kaggle/input/crash-1500/Crash-1500/000001.mp4
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!sudo apt-get update
!sudo apt-get install ffmpeg

Get:1 https://packages.cloud.google.com/apt gcsfuse-focal InRelease [1225 B]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]      
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]        
Get:5 https://packages.cloud.google.com/apt cloud-sdk InRelease [1616 B]       
Get:6 https://packages.cloud.google.com/apt gcsfuse-focal/main amd64 Packages [24.8 kB]
Hit:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease  
Get:8 https://packages.cloud.google.com/apt cloud-sdk/main amd64 Packages [3124 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [3836 kB]
Get:10 https://packages.cloud.google.com/apt cloud-sdk/main all Packages [1485 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [4302 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [3805 kB]
Get:13 http://securi

In [2]:
import pandas as pd

csv_file_path = '/kaggle/input/preprocessed-df/preprocessed_df (1).csv'
annotations = pd.read_csv(csv_file_path)

class_counts = annotations['Severity of the Crash'].value_counts()
print(class_counts)

Severity of the Crash
Moderate    622
Minor       348
Severe      244
moderate     70
severe       57
minor        31
fatal        25
Fatal        18
Name: count, dtype: int64


In [3]:
annotations['Severity of the Crash'] = annotations['Severity of the Crash'].str.lower()
class_counts = annotations['Severity of the Crash'].value_counts()
print(class_counts)

Severity of the Crash
moderate    692
minor       379
severe      301
fatal        43
Name: count, dtype: int64


In [4]:
annotations['Severity of the Crash'] = annotations['Severity of the Crash'].str.replace('fatal', 'severe', case=False)
annotations['label'] = annotations['label'].replace(3, 2)
class_counts = annotations['Severity of the Crash'].value_counts()
print(class_counts)

Severity of the Crash
moderate    692
minor       379
severe      344
Name: count, dtype: int64


In [5]:
unique_labels = annotations['label'].unique()
print(unique_labels)

[1 2 0]


In [ ]:
import os
import pandas as pd

video_dir = '/kaggle/input/crash-1500/Crash-1500'
frames_dir = '/kaggle/working/frames' 

for idx, row in annotations.iterrows():
    video_number = row['Video Number']
    video_path = os.path.join(video_dir, f'{video_number:06}.mp4')
    output_dir = os.path.join(frames_dir, f'{video_number:06}')
    os.makedirs(output_dir, exist_ok=True)

    ffmpeg_command = f'ffmpeg -i "{video_path}" -vf fps=4 "{output_dir}/frame%04d.png"'
#     print(f"Running FFmpeg command: {ffmpeg_command}")
    os.system(ffmpeg_command)
    
    extracted_frames = os.listdir(output_dir)
    print(f"Extracted {len(extracted_frames)} frames for video {video_number}")

In [ ]:
from PIL import ImageFilter,ImageEnhance
import cv2
import numpy as np

def denoise_frame(frame):
    frame = np.array(frame)
    denoised_frame = cv2.medianBlur(frame, 5)
    return Image.fromarray(denoised_frame)

def deblur_frame(frame):
    return frame.filter(ImageFilter.UnsharpMask(radius=2, percent=150, threshold=3))

def enhance_contrast(frame):
    frame = np.array(frame)
    lab = cv2.cvtColor(frame, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl, a, b))
    enhanced_frame = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)
    return Image.fromarray(enhanced_frame)

def sharpen_frame(frame, factor=2.0):
    enhancer = ImageEnhance.Sharpness(frame)
    return enhancer.enhance(factor)

In [ ]:
def compute_optical_flow(prev_frame, next_frame):
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_RGB2GRAY)
    next_gray = cv2.cvtColor(next_frame, cv2.COLOR_RGB2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    return flow

In [ ]:
import os
frames_dir = '/kaggle/working/frames'
video_dirs = next(os.walk(frames_dir))[1]
num_videos = len(video_dirs)

print(f"Number of videos with extracted frames: {num_videos}")

In [ ]:
import torch
import pandas as pd
from torch.utils.data import Dataset
from torchvision.io import read_image
from torchvision import transforms
from PIL import Image
import os
import glob
import torch

class VideoDataset(Dataset):
    def __init__(self, annotations_df, root_dir, transform=None, num_frames=20):
        self.annotations = annotations_df
        self.annotations['Severity of the Crash'] = self.annotations['Severity of the Crash'].str.lower()
        self.root_dir = root_dir
        self.transform = transform
        self.label_map = {'minor': 0, 'moderate': 1, 'severe': 2}
        self.num_frames = num_frames

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        video_number = self.annotations.iloc[idx]['Video Number']
        label = self.label_map[self.annotations.iloc[idx]['Severity of the Crash']]
        frame_dir = os.path.join(self.root_dir, f'{video_number:06}')
        frames = []
        optical_flows = []
        frame_paths = sorted(glob.glob(os.path.join(frame_dir, '*.png')))

        for i, frame_path in enumerate(frame_paths):
            frame = Image.open(frame_path).convert('RGB') 
            frame = denoise_frame(frame)
            frame = sharpen_frame(frame)
            frame = enhance_contrast(frame)
            frame = deblur_frame(frame)

            # Apply transformations
            if self.transform:
                frame = self.transform(frame)
            frame = frame.numpy().transpose(1, 2, 0)  
            frames.append(frame)

            if i > 0:
                prev_frame = frames[i-1]
                next_frame = frames[i]
                flow = compute_optical_flow(prev_frame, next_frame)
                optical_flows.append(flow)

        while len(optical_flows) < len(frames):
            optical_flows.append(np.zeros_like(optical_flows[0]))

        frames = torch.tensor(np.stack(frames)).permute(0, 3, 1, 2)  # Change to (num_frames, channels, height, width)
        optical_flows = torch.tensor(np.stack(optical_flows)).permute(0, 3, 1, 2)  # Change to (num_frames-1, channels, height, width)

        return (frames, optical_flows), label

transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.RandomHorizontalFlip(), 
    transforms.RandomRotation(10), 
    transforms.ToTensor(),  
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = VideoDataset(annotations_df=annotations, root_dir='/kaggle/working/frames', transform=transform)

In [ ]:
import torch
from transformers import TimesformerModel, TimesformerConfig
import torch.nn as nn
import torch.optim as optim

In [ ]:
class VideoClassifier(nn.Module):
    def __init__(self, num_classes, num_frames, num_channels=5):
        super().__init__()
        self.config = TimesformerConfig(
            num_frames=num_frames,
            num_classes=num_classes,
            num_channels=num_channels,
        )
        self.timesformer = TimesformerModel(self.config)
        self.classifier = nn.Linear(self.config.hidden_size, num_classes)
        
    def forward(self, frames, flows):
        batch_size, num_frames, channels, height, width = frames.size()
        flows_padded = torch.zeros((batch_size, num_frames, 2, height, width), device=frames.device)
        flows_padded[:, :, :flows.size(2), :, :] = flows
        combined_input = torch.cat((frames, flows_padded), dim=2)  # Concatenate along the channel dimension
        outputs = self.timesformer(combined_input)
        logits = self.classifier(outputs.last_hidden_state[:, 0, :])  
        return logits

model = VideoClassifier(num_classes=3, num_frames=20)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
class_counts = annotations['label'].value_counts()
print(class_counts)

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader, WeightedRandomSampler, SubsetRandomSampler
from collections import Counter

np.random.seed(42)
dataset_size = len(dataset)
indices = list(range(dataset_size))
np.random.shuffle(indices)

train_split = int(np.floor(0.7 * dataset_size))
val_split = int(np.floor(0.2 * dataset_size))
test_split = dataset_size - train_split - val_split

train_indices = indices[:train_split]
val_indices = indices[train_split:train_split + val_split]
test_indices = indices[train_split + val_split:]

print(f"Total dataset size: {dataset_size}")
print(f"Training set size: {len(train_indices)}")
print(f"Validation set size: {len(val_indices)}")
print(f"Test set size: {len(test_indices)}")

train_labels = [annotations['Severity of the Crash'][idx] for idx in train_indices]
class_counts = Counter(train_labels)
total_samples = len(train_labels)
class_weights = {cls: total_samples / count for cls, count in class_counts.items()}
sample_weights = [class_weights[annotations['Severity of the Crash'][idx]] for idx in train_indices]
sample_weights = torch.tensor(sample_weights, dtype=torch.float)

train_sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)
val_sampler = SubsetRandomSampler(val_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = DataLoader(dataset, batch_size=1, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size=1, sampler=val_sampler)
test_loader = DataLoader(dataset, batch_size=2, sampler=test_sampler)

print("Total effective samples in training set:", len(train_loader) * train_loader.batch_size)

In [ ]:
# Verify the class distribution in the training set
class_counts = {0: 0, 1: 0, 2: 0}
for data in train_loader:
    inputs, labels = data
    for label in labels:
        class_counts[label.item()] += 1

print("Class distribution in the training set:")
print(class_counts)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
num_epochs = 4
best_val_acc = 0.0
save_path = '/kaggle/working/best_model.pth'

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for data in train_loader:
        (inputs, flows), labels = data
        inputs, flows, labels = inputs.to(device), flows.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs, flows)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in val_loader:
            (inputs, flows), labels = data
            inputs, flows, labels = inputs.to(device), flows.to(device), labels.to(device)
            outputs = model(inputs, flows)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_acc = 100 * correct / total

    print('[%d] loss: %.3f, val_acc: %.3f' % (epoch + 1, running_loss / len(train_loader), val_acc))

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), save_path)
        print(f"New best model saved with val_acc: {best_val_acc:.3f}")

In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        (inputs,flows), labels = data
        inputs, flows,labels = inputs.to(device),flows.to(device), labels.to(device)
        outputs = model(inputs,flows)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
test_acc = 100 * correct / total
print('Test Accuracy: %.3f' % test_acc)

In [ ]:
model.load_state_dict(torch.load('/kaggle/working/best_model.pth'))
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        (inputs,flows), labels = data
        inputs, flows,labels = inputs.to(device),flows.to(device), labels.to(device)
        outputs = model(inputs,flows)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_acc = 100 * correct / total
print('Test Accuracy: %.3f' % test_acc)

In [ ]:
!pip install GPUtil

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()                           


In [ ]:
# import torch
# from transformers import VideoMAEConfig, VideoMAEImageProcessor, VideoMAEForVideoClassification
# import torch.nn as nn
# import torch.optim as optim

# class VideoClassifier(nn.Module):
#     def __init__(self, num_classes, num_frames):
#         super().__init__()
#         self.num_frames = num_frames
#         self.image_processor = VideoMAEImageProcessor.from_pretrained("MCG-NJU/videomae-base")
#         self.config = VideoMAEConfig.from_pretrained("MCG-NJU/videomae-base")
#         self.config.num_frames = num_frames  # Set the number of frames in the configuration
#         self.config.num_labels = num_classes  # Ensure the number of classes is set here
#         self.model = VideoMAEForVideoClassification.from_pretrained("MCG-NJU/videomae-base", config=self.config)

#     def forward(self, x):
#         x = x.squeeze(0)
#         x = x.permute(1, 0, 2, 3)
        
#         # Forward pass through VideoMAEForVideoClassification model
#         outputs = self.model(pixel_values=x.unsqueeze(0))
#         return outputs.logits


# model = VideoClassifier(num_classes=3, num_frames=20)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)